In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.cluster import KMeans
from sklearn.gaussian_process.kernels import Matern, RBF
from sklearn.ensemble import RandomForestRegressor

import plotly.express as px

import plotly.graph_objects as go

from scipy.linalg import toeplitz, block_diag

from spe.mse_estimator import ErrorComparer
from spe.data_generation import gen_rbf_X, gen_matern_X, create_clus_split, gen_cov_mat
from spe.forest import BlurredForest
from spe.estimators import kfoldcv, kmeanscv, better_test_est_split, cp_rf_train_test, cp_general_train_test, bag_kfoldcv, bag_kmeanscv

import os

In [2]:
niter = 50

n=30**2
p=30
s=30

delta = 0.

n_estimators = 100
max_depth = 6

snr = 0.4

noise_kernel = 'matern'
noise_length_scale = 1.
noise_nu = 0.5

X_kernel = 'matern'
X_length_scale = 1.
X_nu = 0.5

idx = -1

savedir='~'

In [3]:
# noise_kernel = kernel
# X_kernel = kernel

# noise_length_scale = length_scale
# X_length_scale = length_scale

# noise_nu = nu
# X_nu = nu

In [5]:
if not os.path.exists(os.path.expanduser(savedir)):
    os.makedirs(os.path.expanduser(savedir))

params = pd.DataFrame({'niter': niter,
                       'n': n, 
                       'p': p, 
                       's': s,
                       'snr': snr,
                       'n_estimators': n_estimators,
                       'max_depth': max_depth,
                       'delta': delta,
                       'nk': noise_kernel, 
                       'nls': noise_length_scale, 
                       'nn': noise_nu, 
                       'xk': X_kernel,
                       'xls': X_length_scale,
                       'xn': X_nu}, index=[idx])
params.to_csv(os.path.expanduser(savedir + 'params.csv'))
dffp = os.path.expanduser(savedir + "err_df.csv")
# barfp = os.path.expanduser(savedir + 'barchart.jpeg')

In [6]:
err_cmp = ErrorComparer()

In [7]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, 10, nx)
ys = np.linspace(0, 10, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()
coord = np.stack([c_x, c_y]).T

In [8]:
if noise_kernel == 'rbf':
    Sigma_t = gen_cov_mat(c_x, c_y, RBF(length_scale=noise_length_scale))
elif noise_kernel == 'matern':
    Sigma_t = gen_cov_mat(c_x, c_y, Matern(length_scale=noise_length_scale, nu=noise_nu))
else:
    Sigma_t = np.eye(n)
    
Sigma_t = delta*Sigma_t + (1-delta)*np.eye(n)

if noise_kernel == 'rbf' or noise_kernel == 'matern':
    Chol_t = np.linalg.cholesky(Sigma_t)
else:
    Chol_t = np.eye(n)

In [4]:
models = [RandomForestRegressor(n_estimators=n_estimators,
                                max_depth=max_depth),
          BlurredForest(n_estimators=n_estimators,  
                        max_depth=max_depth, 
                        bootstrap_type=None),
          BlurredForest(n_estimators=n_estimators,  
                        max_depth=max_depth, 
                        bootstrap_type='blur'),
          BlurredForest(n_estimators=n_estimators,  
                        max_depth=max_depth, 
                        bootstrap_type='blur'),
          BlurredForest(n_estimators=n_estimators,  
                        max_depth=max_depth, 
                        bootstrap_type='blur')]

ests = [better_test_est_split,
        better_test_est_split,
        better_test_est_split,
        better_test_est_split,
        better_test_est_split]

est_kwargs = [{'alpha':None},
              {'full_refit': True},
              {'full_refit':False,
               'chol_eps': Chol_t},
              {'full_refit':True,
               'chol_eps': Chol_t},
              {'full_refit':True,
               'chol_eps': np.eye(n)}]

In [9]:
if X_kernel == 'rbf':
    X = gen_rbf_X(c_x, c_y, p)
elif X_kernel == 'matern':
    X = gen_matern_X(c_x, c_y, p, length_scale=X_length_scale, nu=X_nu)
else:
    X = np.random.randn(n,p)

beta = np.zeros(p)
idx = np.random.choice(p,size=s)
beta[idx] = np.random.uniform(-1,1,size=s)

In [10]:
(vrf_err,
 rff_err,
 bfw_err,
 bff_err,
 bfi_err) = err_cmp.compare(models,
                             ests,
                             est_kwargs,
                             niter=niter,
                             n=n,
                             p=p,
                             s=p,
                             snr=snr, 
                             X=X,
                             beta=beta,
                             coord=coord,
                             Chol_t=Chol_t,
                             Chol_s=None,
                             tr_idx=None,
                             fair=False,
                             )

0
0.5833333333333334
Chol_t [[ 5.25154607  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          5.25154607  0.         ...  0.          0.
   0.        ]
 [ 0.          0.          5.25154607 ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  5.25154607  0.
   0.        ]
 [ 0.          0.          0.         ... -0.          5.25154607
   0.        ]
 [ 0.          0.          0.         ... -0.         -0.
   5.25154607]]
Chol_t [[ 5.25154607  0.          0.         ...  0.          0.
   0.        ]
 [ 0.          5.25154607  0.         ...  0.          0.
   0.        ]
 [ 0.          0.          5.25154607 ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  5.25154607  0.
   0.        ]
 [ 0.          0.          0.         ... -0.          5.25154607
   0.        ]
 [ 0.          0.          0.         ... -0.         -0.
   5.25154607]]
Chol_t [[ 5.25154607  0.          0.         ...  0.         

KeyboardInterrupt: 

In [ ]:
risk_vrf = vrf_err.mean()
risk_rff = rff_err.mean()
risk_bfw = bfw_err.mean()
risk_bff = bff_err.mean()
risk_bfi = bfi_err.mean()

In [ ]:
save_df = pd.DataFrame({'VRF': (vrf_err.T),
                        'RF_FR': (rff_err.T),
                        'BF_WR': (bfw_err.T),
                        'BF_FR': (bff_err.T),
                        'BFI_FR': (bfi_err.T)})
save_df.to_csv(dffp)

In [ ]:
df = pd.DataFrame({'VRF': (vrf_err.T),
                   'RF_FR': (rff_err.T),
                   'BF_WR': (bfw_err.T),
                   'BF_FR': (bff_err.T),
                   'BFI_FR': (bfi_err.T)})

In [ ]:
(df).mean()

In [ ]:
df.var()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Control',
    x=['VRF', 'RF_FR', 'BF_WR', 'BF_FR', 'BFI_FR'], y=(df).mean(),
    marker_color=px.colors.qualitative.Plotly,
    text=np.around((df).mean(),3),
    textposition='outside',
    error_y=dict(
        type='data',
        color='black',
        symmetric=False,
        array=(df).quantile(.75) - (df).mean(),
        arrayminus=(df).mean() - (df).quantile(.25))
#         array=(df).mean() + (df).std(),
#         arrayminus=(df).mean() - (df).std())
))
# fig.add_trace(go.Bar(
#     name='Experimental',
#     x=['GenCp', 'KFCV', 'SPCV'], y=(df).mean(),
#     error_y=dict(type='data', array=[1, 2])
# ))
# fig.update_layout(barmode='group')
# fig.add_hline(y=1., line_color='red')
fig.update_layout(
    title=f"FC_\u03B4{delta}_snr{snr}_nk{noise_kernel}_nls{noise_length_scale}_nv{noise_nu}_Xk{X_kernel}_Xls{X_length_scale}_Xv{X_nu}",
    xaxis_title="Method",
    yaxis_title="MSE",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     )
)
barfp = os.path.expanduser(savedir + fig.layout.title['text'] + ".jpeg")
fig.write_image(os.path.expanduser(barfp))
fig.show()